# Asset Correlation



Here we will investigate the correlation between assets, Bitcoin (BTC), S&P500 stocks (SPX) and Gold (GLD).

*Overview:*
* Pull data from Yahoo Finance
* Compute Pearsons Product Moment Coefficient
* Group data over different periods
* Compute once more correlation to see how it changes over time

*Note:* 
* For SPX we have over 90 years of data, while for BTC we have only a little over 5 years, such is it's immaturity.

*Results:*
* Overall we see a weak postive correlation between SPX and GLD
  * On an annual basis, this is supported by the long term trend
* Overall we see a strong positive correlation between BTC and GLD
  * On an annual basis, this is not a supported 
* Overall we see a strong postive correlation between SPX and BTC
  * On an annual basis, BTC flips every two years between a weak to strong positive correlation with SPX.
  * Recently on a monthly basis, we see the strong positive correlation between SPX and BTC.




This notebook supports the following blog post: 

https://siliconmeetssociety.wordpress.com/2020/04/19/correlation/

In [0]:
#@title
#@imports
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [0]:
#@title
#@data sources
btc_data_url="https://query1.finance.yahoo.com/v7/finance/download/BTC-USD?period1=1410912000&period2=1587168000&interval=1d&events=history"
spx_data_url="https://query1.finance.yahoo.com/v7/finance/download/%5EGSPC?period1=-1325635200&period2=1587168000&interval=1d&events=history"
gld_data_url="https://query1.finance.yahoo.com/v7/finance/download/GC=F?period1=951696000&period2=1587168000&interval=1d&events=history"

In [0]:
#@title
#@data loading
btc_data_response = requests.get(btc_data_url)
spx_data_response = requests.get(spx_data_url)
gld_data_response = requests.get(gld_data_url)
btc_df = pd.read_csv(btc_data_url, index_col="Date", usecols=["Date", "Close"], parse_dates=True)
btc_df.columns = ["BTC"]
spx_df = pd.read_csv(spx_data_url, index_col="Date", usecols=["Date", "Close"], parse_dates=True)
spx_df.columns = ["SPX"]
gld_df = pd.read_csv(gld_data_url, index_col="Date", usecols=["Date", "Close"], parse_dates=True)
gld_df.columns = ["GLD"]

In [0]:
#@title
#@data processing
data_df = pd.concat([btc_df, spx_df, gld_df], axis=1)
corr_df = data_df.corr(method="pearson")

In [0]:
#@title
#@Correlation Overall
fig = go.Figure(data=go.Heatmap(
                   z=corr_df.values,
                   x=corr_df.columns,
                   y=corr_df.columns,
                   hoverongaps = False,
                   colorscale='RdBu',
                   zmid=0))
fig.update_layout(title="BTC / SPX / GLD Correlation - Overall")
fig.show()

* Overall we see a weak postive correlation between SPX and GLD
* Overall we see a strong positive correlation between BTC and GLD
* Overall we see a strong postive correlation between SPX and BTC

In [0]:
#@title
#@Plot Zones

high_corr_zones = [
                   dict(
                        type="rect",
                        xref="paper",
                        yref="y",
                        x0=0,
                        y0=0.6,
                        x1=1,
                        y1=1,
                        fillcolor="LightSalmon",
                        opacity=0.5,
                        layer="below",
                        line_width=0,
                    ),
                    dict(
                        type="rect",
                        xref="paper",
                        yref="y",
                        x0=0,
                        y0=-0.6,
                        x1=1,
                        y1=-1,
                        fillcolor="LightSalmon",
                        opacity=0.5,
                        layer="below",
                        line_width=0,
                    )
                ]



In [0]:
#@title
#@Annual Correlation
yearly_corr = data_df.groupby(pd.Grouper(freq='Y')).corr()
btc_index = yearly_corr.index.get_level_values(1)[:3].values.tolist().index("BTC")
spx_index = yearly_corr.index.get_level_values(1)[:3].values.tolist().index("SPX")
gld_index = yearly_corr.index.get_level_values(1)[:3].values.tolist().index("GLD")

fig = go.Figure(
        go.Scatter(x=yearly_corr.iloc[btc_index::3]["SPX"].dropna().index.get_level_values(0).to_pydatetime(),
                   y=yearly_corr.iloc[btc_index::3]["SPX"].dropna().values) 
        )
fig.update_yaxes(range=[-1,1])
fig.update_layout(title="BTC / SPX Correlation - Annual Basis",
                  shapes=high_corr_zones)
fig.show()

* BTC flips every two years between a weak to strong positive correlation with SPX

In [0]:
#@title
#@Annual Correlation - Continued
fig = go.Figure(
        go.Scatter(x=yearly_corr.iloc[btc_index::3]["GLD"].dropna().index.get_level_values(0).to_pydatetime(),
                   y=yearly_corr.iloc[btc_index::3]["GLD"].dropna().values) 
        )
fig.update_yaxes(range=[-1,1])
fig.update_layout(title="BTC / GLD Correlation - Annual Basis",
                  shapes=high_corr_zones)
fig.show()

* Only recently does BTC and GLD have a strong positive correlation

In [0]:
#@title
#@Annual Correlation - Continued
fig = go.Figure(
        go.Scatter(x=yearly_corr.iloc[gld_index::3]["SPX"].dropna().index.get_level_values(0).to_pydatetime(),
                   y=yearly_corr.iloc[gld_index::3]["SPX"].dropna().values) 
        )
fig.update_yaxes(range=[-1,1])
fig.update_layout(title="SPX / GLD Correlation - Annual Basis",
                  shapes=high_corr_zones)
fig.show()

* SPX and GLD does not maintain the strong positive correlation

In [0]:
#@title
#@Monthly Correlation
monthly_corr = data_df['2019-10-01':].groupby(pd.Grouper(freq='M')).corr()
btc_index = monthly_corr.index.get_level_values(1)[:3].values.tolist().index("BTC")

fig = go.Figure(
        go.Scatter(x=monthly_corr.iloc[btc_index::3]["SPX"].dropna().index.get_level_values(0).to_pydatetime(),
                  y=monthly_corr.iloc[btc_index::3]["SPX"].dropna().values) 
        )
fig.update_yaxes(range=[-1,1])
fig.update_layout(title="BTC / SPX Correlation - 6 Months - Monthly Basis",
                  shapes=high_corr_zones)
fig.show()

* Recently on a monthly basis, we see the strong positive correlation between SPX and BTC